# 3.6 예제: 레스토랑 리뷰 감성 분류하기

이 프로젝트는 리뷰와 감성 레이블(긍정 or 부정)이 쌍을 이루는 옐프 데이터셋을 사용합니다.  
데이터를 정제하고, 훈련, 검증, 테스트 세트로 나누는 전처리 단계 및 몇가지를 추가로 설명하면서 프로젝트를 진행합니다.

앞으로 매번 사용할 3개의 보조 클래스에 대해서 간단하게 설명해보면,  
1. Vocabularay = 샘플과 타깃의 인코딩에서 설명한 정수와 토큰 매핑을 수행
2. Vectorizer = 어휘 사전을 캡슐화 하고 리뷰 텍스트 같은 문자열 데이터를 훈련과정에서 사요할 수치 벡터로 전환.
3. DataLoader = 개별 벡터 데이터 포인트를 미니 배치로 모으는 역할

# 3.6.1 옐프 리뷰 데이터셋

해당 프로젝트에서는 데이터셋의 훈련 샘플의 10%만 사용합니다. (Light version)   
이렇게 작은 데이터셋을 사용하면 훈련과 테스트가 빨라지지만 전체 데이터셋을 사용할 때 보다 낮은 정확도를 가집니다.

데이터셋을 훈련,검증,테스트용으로 나눌겁니다.  
훈련세트로 모델을 훈련하고, 검증세트로 모델이 얼마나 잘 작동하는지 평가합니다.  
검증 세트를 기반으로 모델을 선택하게 되면 물가피하게 모델이 검증세트에 더 잘 수행되도록 편향되기 때문에 모델이 점차 개선되는지 재평가 해보기 위해서 세번째 세트인 평가세트를 활용해서 이 문제를 해결합니다.

In [2]:
# import 
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [3]:
# 
args = Namespace(
    raw_train_dataset_csv = "raw_train.csv",
    raw_test_dataset_csv = "raw_test.csv",
    proportion_subset_of_train = 0.1,
    train_proportion = 0.7,
    val_proportion = 0.15,
    test_proportion = 0.15,
    output_munged_csv = "reviews_with_splits_lite.csv",
    seed = 1337
)

인자와 부속명령을 위한 명령행 옵션인, argparse를 활용해서 파이썬 내부에서 파일을 다운받고, 데이터를 나누어 줄 수 있습니다.  
train 70%, validation 15%, test 15%로 데이터를 분할할 예정입니다.

In [4]:
# 원본 데이터를 읽습니다.
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

다운받은 원본데이터를 파이썬에서 활용하기 위해 train_reviews 변수에 데이터를 읽어주고 컬럼명을 rating과 review 로 지정해주었다.

In [5]:
# 리뷰 클래스 비율이 동일하도록 만듭니다.
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())
    
review_subset = []

for _, item_list in sorted(by_rating.items()):
    n_total = len(item_list)
    n_subset = int(args.proportion_subset_of_train * n_total)
    review_subset.extend(item_list[:n_subset])
    
review_subset = pd.DataFrame(review_subset)

review_subset에 존재하는 데이터에서 10% 에 해당하는 데이터만 따로 저장해준다.  
그리고 defaultdic메서드를 활용해서 클래스별로 비율이 동일하도록 만들어 준 것이다.

In [6]:
review_subset.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...


In [7]:
train_reviews.rating.value_counts()

1    280000
2    280000
Name: rating, dtype: int64

앞에서 클래스별로 비율이 동일하도록 만들어준 결과값입니다. (클래스별로 각각 28000개)

In [8]:
# 고유 클래스
set(review_subset.rating)

{1, 2}

클래스는 1과 2로 나누어지는 것을 볼 수 있다.  
1은 negative, 2는 positive 클래스이다.

In [9]:
# 코드 3-12 훈련,검증,테스트 세트 만들기

# 별점 기준으로 나누어, 훈련, 검증, 테스트를 만듭니다.
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())
    
# 분할 데이터를 만듭니다.
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)
    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)
    
    # 데이터 포인트에 분할 속성을 추가합니다.
    for item in item_list[:n_train]:
        item['split'] = 'train'
    
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
        
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'
        
    # 최종 리스트에 추가합니다.
    final_list.extend(item_list)
    
# 분할 데이터를 데이터 프레임으로 만듭니다.
final_reviews = pd.DataFrame(final_list)

앞서 나왔던 args에서 지정한 7:1.5:1.5 비율로 각각 데이터를 train/val/test로 지정했고, 최종 리스트에 추가해주었다.

리스트형태의 데이터를 분석에 용이한 pandas 데이터 프레임으로 만들어준다.

In [10]:
final_reviews.split.value_counts()

train    39200
val       8400
test      8400
Name: split, dtype: int64

In [11]:
# 코드 3-13 최소한의 데이터 정제 작업
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_reviews.review = final_reviews.review.apply(preprocess_text)

최소한의 데이터 정제작업을 거친다.  
정규식을 활용하여 기호 앞뒤에 공백을 넣고, 구두점이 아닌 기호를 제거하는 정제작업을 진행해주었다.

In [12]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

In [13]:
final_reviews.head()

,rating,review,split
0,negative,terrible place to work for i just heard a stor...,train
1,negative,"hours , minutes total time for an extremely s...",train
2,negative,my less than stellar review is for service . w...,train
3,negative,i m granting one star because there s no way t...,train
4,negative,the food here is mediocre at best . i went aft...,train


In [15]:
final_reviews.to_csv(args.output_munged_csv, index=False)

# 3.6.2 파이토치 데이터셋 이해하기

해당 프로젝트는 pytorch 프레임워크를 기반으로 진행된다. 그리고 클래스 객체를 활용해서 주요한 파이프라인을 수행하게 된다.  
또한 파이토치는 Dataset, DataLoader를 활용하여 데이터의 형태를 변환시켜준다.

ReviewDataset 클래스는 데이터셋이 최소한으로 정제되고 3개로 나뉘었다고 가정한다.  
특히 해당 데이터셋은 공백을 기준으로 나눠서 토큰 리스트를 얻을 수 있다고 가정합니다.  
샘플이 훈련, 검증, 테스트 중 어느세트에 있는지 표시되었다고 가정한다.

In [17]:
# 임포트

from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

In [18]:
# 코드 3-14 옐프 리뷰 데이터를 위한 파이토치 데이터셋 클래스

class ReviewDataset(Dataset):
    # 클래스 인스턴스 생성시 초기화 하면서 실행되는 부분
    # self란? 인스턴스(클래스에 의해 만들어진 객체) 자기 자신을 의미,
    # self 가 있는 것이 인스턴스 변수
    def __init(self, review_df, vectorizer):
        """
        매개변수 : 
            review_df (pands.DataFrame) : 데이터셋
            vectorizer (ReviewVectorizer) : ReviewVectorizer 객체
        """
        self.review_df = review_df
        self._vectorizer = vectorizer
        
        self.train_df = self.review_df[self.review_df.split == 'train']
        self.train_size = len(self.train_df)
        
        self.val = self.review_df[self.review_df.split == 'val']
        self.validation_size = len(self.val_df)
        
        self.test_df = self.review_df[self.review_df.split == 'test']
        self.test_size = len(self.test_df)
        
        self._lookup_dict = {'train' : (self.train_df, self.train_size),
                             'val' : (self.val_df, self.validation_size),
                             'test' : (self.test_df, self.test_size)}
        
        self.set_spilt('train')
    
    # 클래스 메소드 (데코레이터 사용) = 클래스 변수를 컨트롤 할 때 사용된다.
    # cls 인자를 받음. cls란? ReviewDataset 클래스를 뜻함.
    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만듭니다.
        캐시된 ReviewVectorizer 객체를 재사용할 때 사용합니다.
        
        매개변수:
            review_csv (str): 데이터셋의 위치
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장위치
        반환값:
            ReviewDataset의 인스턴스
        """
        
        review_df = pd.read_csv(review_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(review_df, vectorizer)
    
    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """파일에서 Reviewvectorizer 객체를 로드하는 정적 메서드
        
        매개변수:
            vectorizer_filepath (str) : 직렬화된 ReviewVectorizer 객체의 위치
        반환값:
            ReviewVectorizer의 인스턴스
        """
        with open(vectorizer_filepath) as fp:
            return ReviewVectorizer.from_serializable(json.load(fp))
    
    def save_vectorizer(self, vectorizer_filepath):
        """ReviewVectorizer 객체를 json 형태로 디스크에 저장합니다.
        
        매개변수:
            vectorizer_filepath (str) : ReviewVectorizer 객체의 저장위치
        """
        self._target_split = split
        self._traget_df, self._target_size = self._lookup_dict[split]
        
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """파이토치 데이터셋의 주요 진입 메서드
        
        매개변수:
            index (int): 데이터 포인트의 인덱스
        반환값:
            데이터 포인트의 특성(x_data)과 레이블(y_target)로 이루어진 딕셔너리
        """
        row = self._target_df.iloc[index]
        
        review_vector = \
            self._vectorizer.vectorize(row.review)
        
        rating_index = \
            self._vectorizer.rating_vocab.lookup_token(row.rating)
        
        return {'x_data' : review_vector,
                 'y_target' : rating_index}
    
    def get_num_batches(self, batch_size):
        """배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다.
        
        매개변수:
            batch_size (int)
        반환값:
            배치 개수
        """
        return len(self) // batch_size

1. 파이토치에서 새로운 데이터셋을 사용하려면 먼저 Dataset 클래스를 상속하여 __getitem__(), __len__() 메서드를 구현해야한다.  
from torch.utils.data import Dataset, DataLoader  
2. 해당 클래스 안에는 다양한 파이토치 유틸리티를 사용하는데, DataLoader / ReviewVectorizer과 같은 클래스는 아래에 나온다. (클래스들은 서로 크게 의존한다.)  
    ReviewVectorizer => 리뷰텍스트를 수치로 변환하는 클래스
    DataLoader => 데이터셋에서 샘플링하고 모아서 미니배치를 만드는 클래스

### Dataset 클래스를 상속한다는 뜻이 무엇일까?

torch.utils.data.Dataset은 데이터셋을 나타내는 '추상 클래스'이다.  
우리만의 데이터셋은 Dataset에 상속하고, 아래와 같이 오버라이드 해야한다.  
    len(dataset)에서 호출되는 __len__은 데이터셋의 크기를 리턴한다.  
    dataset[i]에서 호출되는 __getitem__은 i번째 샘플을 찾는데 사용된다.  
    즉, __init__을 통해 데이터를 처리해서 가져오지만, __getitem__을 이용해서 데이터를 하나씩 판독한다.  
    그리고 최정 리턴값은{'x_data':review_vector,'y_target':rating_index} 형태의 사전 형태를 가진다.

### ReviewDataset 커스텀 데이터셋에 등장하는 함수 정리

1. __init__(self, review_df,Vectorizer)  
    데이터셋(review_df)  
    벡터화해주는 객체 (vectorizer)  
    train_df/train_size/val_df/val_size/test_df/test_size 변수를 정의

2. @classmethod load_dataset_and_make_vectorizer(cls, review_csv)  
    데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만들어주는 함수  
    review_csv라는 데이터셋의 위치를 매개변수로 받는다.  
    ReviewDataset(review_df, ReviewVectorizer.from_dataset(train_review_df)한 결과값을 반환  
    reviewVectorizer.from_dataset는 데이터셋 데이터프레임에서 Vectorizer객체를 만드는 메서드  
    반환값이 ReviewVectorizer 객체이다.
    
3. @staticmethod load_vectorizer_only(vectorizer_filepath)  
    파일에서 ReviewVectorizer 객체를 로드하는 정적 메서드  
    ReviewVectorizer.from_serializable(json.load(fp))를 반환한다.  
    ReviewVectorizer.from_serializable는 직렬화된 딕셔너리(json.load(fp))에서 ReviewVectorizer 객체를 만드는 메소드  
    
4. save_vectorizer(seef, vectorizer_filepath)  
    ReviewVectorizer 객체를 json 형태로 디스크에 저장하는 메소드  

5. get_vectorizer(self)  
    벡터 변환 객체를 반환하는 메서드 self._vectorizer 값을 반환
    
6. set_spilt(self, split="train")  
    데이터 프레임에 있는 열을 사용하여 분할 세트를 선택  
    "train", "test","val" 중 하나  

7. __len__(self)  
8. __getitem__(self, index)  
    {"x_data" : review_vector, "y_target" : rating_index}로 이루어진 dic 형태로 반환  
    
9. get_num_batches(self, batch_size)  
    배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환

# 3.6.3 Vocabulary, Vectorizer, DataLoader 클래스  

해당 예제는 위 세개의 클래스를 활용하여 중요한 파이프라인을 수행합니다.  
텍스트의 입력을 벡터의 미니배치로 바꿔주기 위해 해당 클래스들을 사용합니다.  
이 세개의 클래스는 각 토큰을 정수에 매핑하고, 이 매핑을 각 데이터포인트에 적용하여 벡터 형태로 변환해줍니다. 그 다음 벡터로 변환하 데이터 포인트를 모델을 위해 미니배치로 모은다.  
전처리 된 데이터(텍스트)를 사용한다. (즉 데이터 포인트는 토큰화 된 집합이다.)

### Vocabulary 클래스

머신러닝 파이프라인에 필요한 토큰과 정수 매핑을 관리하는 클래스  
토큰과 정수로 매핑하는 크래스 (텍스트의 배치를 미니배치로 바꾸는 과정의 첫 단계)  
토큰과 정수 사이를 1:1 매핑하는 방법 (반대로 매핑하는 경우까지, 총 딕셔너리 두개 필요)  
두 딕셔너리를 Vocabulary 클래스에 캡슐화 한 것  
훈련 데이터셋에 없는 단어는 UNK(Unknown)으로 처리  
자주 등장하지 않는 토큰들을 제한해줌 => 이런 토큰들이 UNK로 처리되는것

In [19]:
class Vocabulary(object):
    """매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스 """
    
    def __init(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        매개변수:
            token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
            add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
            unk_token (str): Vocabulary에 추가할 UNK 토큰
        """
        
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_token = token_to_idx
        
        self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    
    def to_serializable(self):
        """ 직렬화할 수 있는 딕셔너리를 반환합니다."""
        return {'token_to_idx': self._token_to_idx,
                 'add_unk': self._add_unk,
                 'unk_token': self._unk_token}
    
    @classmethod
    def from_serializable(cls, contents):
        """직렬화 된 딕셔너리에서 Vocabulary 객체를 만듭니다."""
        return cls(**contents)
    
    # 새로운 토큰을 추가하기 위한 함수
    def add_token(self, token):
        """토큰을 기반으로 매핑 딕셔너리를 업데이트합니다.
        
        매개변수:
            token (str): Vocabulary에 추가할 토큰
        반환값:
            index (int): 토큰에 상응하는정수
            """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        """토큰 리스트를 Vocabulary에 추가합니다.
        매개변수:
            tokens (list): 문자열 토큰 리스트
        반환값:
            indices (list): 토큰 리스트에 상응되는 인덱스 리스트
        """
        return [self.add_token(token) for token in tokens]
    
    # 토큰에 해당하는 인덱스를 추출하기 위한 함수
    def lookup_token(self, token):
        """토큰에 대응하는 인덱스를 추출합니다.
        토큰이 없으면 UNK 인덱스를 반환합니다.
        
        매개변수:
            token (str): 찾을 토큰
        반환값:
            index (int): 토큰에 해당하는 인덱스
        노트:
            UNK 토큰을 사용하려면 (Vocabulary에 추가하기 위해)
            'unk_index'가 0보다 커야 합니다.
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
        
    #인덱스에 해당하는 토큰을 추출하기 위한 함수
    def lookup_ind(self, index):
        """인덱스에 해당하는 토큰을 반환합니다.
        
        매개변수:
            index (int): 찾을 인덱스
        반환값:
            token (str): 인덱스에 해당하는 토큰
        에러:
            keyError: 인덱스가 Vocabulary에 없을 때 발생합니다.
        """
        if index not in self._idx_to_token:
            raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
        return self.idx_to_token[index]
    
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    
    def __len__(self):
        return len(self._token_to_idx)

### Vocabulary 클래스에 등장하는 함수 정리

1. __init__(self, token_to_idx=None, add_unk=True, unk_token="UNK"  
    1. 매개변수 설명  
    * token_to_idx(dict): 기존 토큰-인덱스 매핑 딕셔너리  
    * add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그  
    * unk_token (str): Vocabulary에 추가할 UNK 토큰  
    
2. to_serializable(self)  
    1. 직렬화 할 수 있는 딕셔너리를 반환  
    * {'token_to_idx': self._token_to_idx,'add_unk': self._add_unk, 'unk_token': self._unk_token}형태  
    
3. @classmethod form_serializable(cls, contents)  
    1. 직렬화된 딕셔너리에서 vocabulary 객체를 생성  
    
4. add_token(self, token)  
    1. 새로운 토큰을 추가하기 위한 함수  
    2. 토큰을 기반으로 매핑 딕셔너리를 업데이트해준다.  
    3. 매개변수 token이 Vocabulary에 추가할 토큰이 된다.
    4. return index: 토큰에 상응하는 인덱스가 반환값으로 출력
    
5. add_mamy(self, tokens)  
    1. 토큰 리스트를 vocabulary에 추가  
    2. tokens는 문자열 list
    3. 반환값도 tokens에 상응하는 인덱스 값
    
6. lookup_token(self,token)  
    1. 토큰에 댕ㅇ하는 인덱스를 추출하는 메서드  
    2. 매개변수 token이 찾을 토큰이고  
    3. 반환값은 token이 갖는 인덱스값
    
7. lookup_index(self, index)  
    1. 인덱스에 대응하는 토큰을 찾는 메소드  
    2. 매개변수 index가 찾을 인덱스  
    3. 반환값은 index에 해당하는 토큰